%md

# 🎬 Projeto de Análise de Dados - IMDB

## 📊 Notebook 5 — Análise dos Dados e Respostas às Perguntas

Neste notebook, realizamos a etapa de **análise exploratória final** da base de dados tratada. Com os dados já estruturados, limpos e organizados em camadas (bronze, silver e gold), aqui buscamos **responder às perguntas principais definidas no projeto**.

Cada pergunta será respondida com seu respectivo contexto, após a consulta SQL.

------

##Serão 10 tópicos abordados. São eles:

- 1 - Os 10 Títulos de Cinema Mais Votados e com Melhores Avaliações
- 2 - Diretores com mais Votos e com as Melhores Médias de Avaliação
- 3 - Os países com o maior número de lançamentos de filmes nos últimos 30 anos
- 4 - Evolução de Lançamentos (Cinema) por Década
- 5 - Número de Atrizes (únicas) por Década (até 2019)
- 6 - Proporção Entre Atores (únicos) e Atrizes (únicas) ao Longo das Décadas
- 7 - Média de Duração de Tempo dos Filmes ao Longo das Décadas 
- 8 - Diretores menos prolíficos com alta média de avaliação
- 9 - Gêneros de Cinema ao Longo das Décadas
- 10 - Evolução da Produção de Séries de TV (2010–2024)

## 🎬⭐ 1 - Os 10 Títulos de Cinema Mais Votados e com Melhores Avaliações


In [0]:
%sql
SELECT
  tb.primaryTitle AS titulo,
  tb.startYear AS ano,
  tr.numVotes AS total_votos,
  tr.averageRating AS avaliacao_media
FROM gold_title_ratings tr
JOIN gold_title_basics tb ON tr.tconst = tb.tconst
WHERE tb.titleType = 'movie'
  AND tr.numVotes IS NOT NULL
  AND tr.averageRating IS NOT NULL
ORDER BY tr.numVotes DESC, tr.averageRating DESC
LIMIT 10;



titulo,ano,total_votos,avaliacao_media
The Shawshank Redemption,1994,3029407,9.3
The Dark Knight,2008,3005759,9.0
Inception,2010,2670355,8.8
Fight Club,1999,2450435,8.8
Forrest Gump,1994,2367616,8.8
Interstellar,2014,2326479,8.7
Pulp Fiction,1994,2324367,8.9
The Matrix,1999,2145889,8.7
The Godfather,1972,2114342,9.2
The Lord of the Rings: The Fellowship of the Ring,2001,2098666,8.9


*Análise da consulta SQL estará após a análise SQL da pergunta 3.

## 🎥⭐ 2 - Diretores com mais Votos e com as Melhores Médias de Avaliação


*Análise da consulta SQL estará após a análise SQL da pergunta 3.

In [0]:
%sql
SELECT
  nb.primaryName AS diretor,
  COUNT(tb.tconst) AS total_filmes,
  SUM(tr.numVotes) AS total_votos,
  ROUND(AVG(tr.averageRating), 2) AS media_avaliacao
FROM gold_title_crew tc
JOIN gold_title_basics tb ON tc.tconst = tb.tconst
JOIN gold_title_ratings tr ON tb.tconst = tr.tconst
JOIN gold_name_basics nb ON split(tc.directors, ',')[0] = nb.nconst
WHERE tb.titleType = 'movie'
GROUP BY nb.primaryName
HAVING total_votos > 0
ORDER BY total_votos DESC
LIMIT 10;

diretor,total_filmes,total_votos,media_avaliacao
Christopher Nolan,12,17135654,8.17
Steven Spielberg,35,15049279,7.31
Quentin Tarantino,12,11313974,7.87
Martin Scorsese,38,11102632,7.45
Ridley Scott,29,9558774,6.96
David Fincher,12,9508005,7.58
Peter Jackson,15,9289624,7.7
Robert Zemeckis,22,7843561,6.99
James Cameron,11,6796318,7.28
Tim Burton,20,5963403,6.99


## 🎬🌍 3 - Os países com o maior número de lançamentos de filmes nos últimos 30 anos
Lançamentos gerais, e não propriamente produções originais de cada país

In [0]:
%sql
SELECT
  ta.region AS pais,
  COUNT(DISTINCT tb.tconst) AS total_filmes
FROM gold_title_basics tb
JOIN gold_title_akas ta ON tb.tconst = ta.titleId
WHERE tb.titleType = 'movie'
  AND ta.region IS NOT NULL
  AND ta.region NOT IN ('\\N', 'XWW')
  AND tb.startYear >= 1995
GROUP BY ta.region
ORDER BY total_filmes DESC
LIMIT 10;


pais,total_filmes
US,181073
GB,114636
CA,64183
IN,59983
DE,58761
AU,47717
RU,46800
FR,46436
JP,45444
ES,37313


## 🌍 A Influência Internacional no Cinema

A julgar pelos títulos e pelos nomes dos diretores levantados, as duas primeiras perguntas mostram um predomínio óbvio — e já esperado — do cinema americano na produção audiovisual mundial.

Tentei portanto explorar os dados em busca de um entendimento sobre a carga cultural e a participação dos países na indústria cinematográfica: **Quem produz mais cinema no mundo, depois dos EUA?**

O problema:
> ❌ Não há dados sobre o país de origem da produção dos filmes  
> ❌ Nem sobre o local de nascimento dos diretores ou profissionais

O campo `region`, presente na base de títulos alternativos (`title.akas`), **não representa o país de origem da obra**, mas sim o **país onde o título foi lançado**. Por isso, um mesmo filme pode aparecer associado a múltiplas regiões.

---

## 🔎 Estratégia alternativa

Para contornar essa limitação e ainda assim mensurar a presença global no circuito cinematográfico, a abordagem adotada foi:

➡️ Medir a quantidade de títulos lançados por região a partir de 1995. 

➡️ Usar isso como indicador de consumo e inserção internacional no mercado audiovisual.

---

## 📊 Conclusões preliminares

- Há um predomínio massivo do Ocidente: dos 10 países com mais lançamentos, apenas  **Índia** e **Japão** representam o Oriente.

- A **população não é um fator determinante**: a Índia, o país mais populoso do mundo e com forte presença cinematográfica via Bollywood, aparece apenas na **última posição**.

A tabela abaixo mostra uma razão entre filmes por milhão de habitantes, por país:

## 🎬 Produção de Filmes por Milhão de Habitantes

A tabela abaixo apresenta os **10 países com maior número absoluto de filmes lançados** segundo a base do IMDB, acompanhada da **população estimada** de cada país e a taxa proporcional de **filmes por milhão de habitantes**.

| País                    | População Estimada | Total de Filmes (1995–2024) | Filmes por Milhão de Habitantes |
|-------------------------|---------------------|------------------------------|----------------------------------|
| 🇦🇺 Austrália (AU)       | 26.000.000           | 47.717                       | **1.835**                        |
| 🇬🇧 Reino Unido (GB)     | 67.000.000           | 114.636                      | **1.711**                        |
| 🇨🇦 Canadá (CA)          | 38.000.000           | 64.183                       | **1.689**                        |
| 🇪🇸 Espanha (ES)         | 47.000.000           | 37.313                       | **793**                          |
| 🇫🇷 França (FR)          | 65.000.000           | 46.436                       | **714**                          |
| 🇩🇪 Alemanha (DE)        | 83.000.000           | 58.761                       | **708**                          |
| 🇺🇸 Estados Unidos (US)  | 331.000.000          | 181.073                      | **547**                          |
| 🇯🇵 Japão (JP)           | 125.000.000          | 45.444                       | **363**                          |
| 🇷🇺 Rússia (RU)          | 144.000.000          | 46.800                       | **325**                          |
| 🇮🇳 Índia (IN)           | 1.400.000.000        | 59.983                       | **43**                           |


---

Apesar de os Estados Unidos liderarem em número absoluto de filmes lançados desde 1995, países com populações significativamente menores, como **Austrália**, **Reino Unido** e **Canadá**, apresentam uma inserção proporcionalmente maior quando pensamos em filmes por milhão de habitantes. A capacidade de produção cultural está mais ligada à infraestrutura e investimento na indústria do que ao tamanho da população.


### 🌍 Relação entre Produção Cultural e Indicadores Sociais (1995–2024)

Nesta etapa da análise, buscamos entender se existe relação entre o **nível de desenvolvimento social dos países** e a **densidade de produção audiovisual**, medida aqui pela quantidade de filmes lançados por milhão de habitantes.

| País                    | 🎬 Filmes por milhão | 🧠 IDH | 💰 PIB per capita (USD) | ✏️ Alfabetização (%) | 🌐 Acesso à Internet (%) |
|-------------------------|----------------------|--------|--------------------------|-----------------------|---------------------------|
| 🇦🇺 Austrália (AU)       | 1.835                | 0.951  | 62.000                   | 99%                   | 96%                       |
| 🇬🇧 Reino Unido (GB)     | 1.711                | 0.929  | 47.000                   | 99%                   | 97%                       |
| 🇨🇦 Canadá (CA)          | 1.689                | 0.936  | 52.000                   | 99%                   | 96%                       |
| 🇪🇸 Espanha (ES)         |   793                | 0.905  | 34.000                   | 98%                   | 95%                       |
| 🇫🇷 França (FR)          |   714                | 0.903  | 44.000                   | 99%                   | 95%                       |
| 🇩🇪 Alemanha (DE)        |   708                | 0.942  | 51.000                   | 99%                   | 97%                       |
| 🇺🇸 Estados Unidos (US)  |   547                | 0.921  | 76.000                   | 99%                   | 98%                       |
| 🇯🇵 Japão (JP)           |   363                | 0.925  | 40.000                   | 99%                   | 94%                       |
| 🇷🇺 Rússia (RU)          |   325                | 0.822  | 12.000                   | 99%                   | 85%                       |
| 🇮🇳 Índia (IN)           |    43                | 0.633  |  2.500                   | 77%                   | 50%                       |

---

Ao cruzar os dados de filmes por milhão com indicadores sociais como **IDH**, **PIB per capita** e **acesso à internet**, percebe-se que países com melhores indicadores socioeconômicos tendem a apresentar maior produção inserção na indústria audiovisual proporcionalmente falando. Por outro lado, países como **Índia** e **Rússia**, mesmo com grandes populações e relevância cultural, ficam atrás no ranking.



## 🎞️ 4 - Evolução de Lançamentos (Cinema) por Década

In [0]:
%sql
SELECT
  FLOOR(startYear / 10) * 10 AS decada,
  COUNT(*) AS total_lancamentos
FROM gold_title_basics
WHERE titleType = 'movie'
  AND startYear IS NOT NULL
  AND startYear >= 1910
GROUP BY FLOOR(startYear / 10) * 10
ORDER BY decada;

decada,total_lancamentos
1910,12991
1920,22121
1930,20580
1940,14714
1950,23866
1960,31804
1970,40443
1980,44156
1990,45922
2000,78190


### 🔎 O que observamos?

Ao agrupar os dados por década a partir de 1910, identificamos um crescimento contínuo no número de filmes lançados em quase todas as décadas. No entanto, **duas exceções importantes** se destacam:

- 📉 A **década de 1940** teve o **segundo menor número de lançamentos** desde 1910 — ficando à frente apenas da própria década de 1910.
- 📉 A **década de 1930** também registrou números baixos em comparação com o ritmo crescente observado em outras décadas.

---

Essas quedas **não são aleatórias** e refletem o impacto de eventos históricos de grande escala:

## - Segunda Guerra Mundial (1939–1945)
## - Grande Depressão (1929–1939)
---

### 📈 A retomada

A partir da década de 1950, com o fim do conflito e a reconstrução econômica mundial, a indústria do cinema retomou seu crescimento e seguiu o padrão de crescimento década após decada.

---

### 🧠 Conclusão

A análise da evolução por décadas **reforça a sensibilidade da indústria audiovisual a fatores históricos e geopolíticos**. 


##5 - 🎬 Número de Atrizes (únicas) por Década (até 2019)

In [0]:
%sql
SELECT 
  FLOOR(tb.startYear / 10) * 10 AS decada,
  COUNT(DISTINCT nb.primaryName) AS total_atrizes
FROM gold_title_principals tp
JOIN gold_name_basics nb ON tp.nconst = nb.nconst
JOIN gold_title_basics tb ON tp.tconst = tb.tconst
WHERE tp.category = 'actress'
  AND tb.startYear IS NOT NULL
  AND tb.startYear >= 1910
  AND tb.startYear < 2020
GROUP BY FLOOR(tb.startYear / 10) * 10
ORDER BY decada;





decada,total_atrizes
1910,10361
1920,12351
1930,13244
1940,12852
1950,26652
1960,44199
1970,59007
1980,79893
1990,120296
2000,271262



![Gráfico da Pergunta 5](https://github.com/plemos66/projeto_mvp_eng_de_dados/raw/main/img/5.png)


## 👩‍🎬 Participação de Atrizes ao Longo das Décadas

Esta análise traz uma perspectiva histórica da presença de atrizes nas produções cinematográficas entre as décadas de 1910 e 2010,

O nome da atriz é contabilizado apenas uma vez por década, mesmo que ela tenha participado de mais de uma produção no mesmo período.

## Observações:

- **Crescimento consistente**: o número de atrizes cresce consistentemente ao longo das décadas, assim como a produção dos filmes.

- **Explosão nos anos 2000**: As décadas de 1990 e 2000 mostram saltos significativos nessa análise, indicando uma indústria mais diversificada e com mais oportunidades de atuação feminina, inclusive em produções seriadas. 

- **Relevância social**: a guinada nos anos 90/2000 reflete a concordância entre os dados expostos e as tendências sociais, que discutem e exigem cada vez mais a participação feminina no mercado de trabalho, independente da área de atuação.

##6 - 👨👩‍🎬 Proporção Entre Atores (únicos) e Atrizes (únicas) ao Longo das Décadas

In [0]:
%sql
WITH totais_por_decada AS (
  SELECT 
    FLOOR(tb.startYear / 10) * 10 AS decada,
    COUNT(DISTINCT nb.primaryName) AS total_decada
  FROM gold_title_principals tp
  JOIN gold_name_basics nb ON tp.nconst = nb.nconst
  JOIN gold_title_basics tb ON tp.tconst = tb.tconst
  WHERE tp.category IN ('actor', 'actress')
    AND tb.startYear IS NOT NULL
    AND tb.startYear >= 1910
    AND tb.startYear < 2020
  GROUP BY FLOOR(tb.startYear / 10) * 10
),

atores_e_atrizes AS (
  SELECT 
    FLOOR(tb.startYear / 10) * 10 AS decada,
    tp.category,
    COUNT(DISTINCT nb.primaryName) AS total_categoria
  FROM gold_title_principals tp
  JOIN gold_name_basics nb ON tp.nconst = nb.nconst
  JOIN gold_title_basics tb ON tp.tconst = tb.tconst
  WHERE tp.category IN ('actor', 'actress')
    AND tb.startYear IS NOT NULL
    AND tb.startYear >= 1910
    AND tb.startYear < 2020
  GROUP BY FLOOR(tb.startYear / 10) * 10, tp.category
)

SELECT 
  a.decada,
  a.category,
  a.total_categoria,
  ROUND((a.total_categoria / t.total_decada) * 100, 2) AS percentual
FROM atores_e_atrizes a
JOIN totais_por_decada t ON a.decada = t.decada
ORDER BY a.decada, a.category;




decada,category,total_categoria,percentual
1910,actor,15562,60.05
1910,actress,10361,39.98
1920,actor,21076,63.14
1920,actress,12351,37.0
1930,actor,23256,63.85
1930,actress,13244,36.36
1940,actor,21669,62.81
1940,actress,12852,37.25
1950,actor,45824,63.28
1950,actress,26652,36.8



![Gráfico da Pergunta 6](https://github.com/plemos66/projeto_mvp_eng_de_dados/raw/main/img/6.png)


##🔎 O que observei
Apesar do crescimento progressivo do número de atrizes trabalhando nas produções audiovisuais, a variação percentual entre homens e mulheres é pequena ao longo do tempo. Mas ela existe e diminui a distància proporcional entre gêneros!


A partir da década de 90, o percentual feminino de presença nos filmes é maior em relação ao período entre 1920 e 1989, o que indica sim um movimento pautado por privilegiar a figura feminina dentro da indústria cinematográfica. 

### 🧠 Conclusão
Os homens ainda são muito mais presentes quando o assunto é ganhar o papel em um filme, mas as mulheres vem não só crescendo sua presença década após década, mas também diminuindo a distância proporcional entre gêneros.

##Obs:
Aqui eu também queria fazer um recorte entre as outras áreas de atuação, como diretores, roteiristas e etc, mas não há nenhum dado qualificando-os por gênero. Para poder dividir assertivamente, só usando as categorias de Actor e Actress, e por isso a análise foi fechada apenas nessa ramo de atuação.

##🕒 7 - Média de Duração de Tempo dos Filmes ao Longo das Décadas 

In [0]:
%sql
SELECT
  FLOOR(startYear / 10) * 10 AS decada,
  ROUND(AVG(runtimeMinutes), 1) AS duracao_media_minutos
FROM gold_title_basics
WHERE titleType = 'movie'
  AND runtimeMinutes IS NOT NULL
  AND runtimeMinutes > 0
  AND startYear IS NOT NULL
  AND startYear >= 1910
GROUP BY FLOOR(startYear / 10)
ORDER BY decada;


decada,duracao_media_minutos
1910,61.6
1920,71.6
1930,80.3
1940,85.8
1950,90.3
1960,92.9
1970,90.5
1980,91.7
1990,92.1
2000,89.2



![Gráfico da Pergunta 7](https://github.com/plemos66/projeto_mvp_eng_de_dados/raw/main/img/7.png)


## 🕒 O Tempo dos Filmes

Até chegar no padrão "moderno" dos filmes, que se deu na década de 50, os filmes tinham uma outra característica, mais curtos, e foram aumentando a minutagem ao longo do tempo.

A partir dos anos 50, os filmes ganham mais ou menos o mesmo padrão de minutagem, tendo o seu pico na década de 60. 

### 🧠 Reflexão

Na era da fugacidade, em que a produção de conteúdo em plataformas como o Tik Tok se limitam a apenas 1 minuto de vídeo, e 8 minutos no Youtube já é muito longo, como estará esse gráfico daqui a 2 décadas? Os filmes acompanharão a tendência das redes sociais e o ritmo da vida social? 


## 🎬 8 - Diretores menos prolíficos com alta média de avaliação

Quem são os que têm menos títulos, porém com média de avaliação alta?

In [0]:
%sql
SELECT 
  nb.primaryName AS diretor,
  COUNT(DISTINCT tr.tconst) AS total_filmes,
  SUM(tr.numVotes) AS total_votos,
  ROUND(AVG(tr.averageRating), 2) AS media_avaliacao
FROM gold_title_crew tc
JOIN gold_title_ratings tr ON tc.tconst = tr.tconst
JOIN gold_name_basics nb ON SPLIT(tc.directors, ',')[0] = nb.nconst
JOIN gold_title_basics tb ON tc.tconst = tb.tconst
WHERE tb.titleType = 'movie'
  AND tr.averageRating IS NOT NULL
  AND tr.numVotes >= 10000
  AND tc.directors IS NOT NULL
GROUP BY nb.primaryName
HAVING total_filmes BETWEEN 2 AND 5
ORDER BY media_avaliacao DESC
LIMIT 10;


diretor,total_filmes,total_votos,media_avaliacao
Slobodan Sijan,2,34845,8.75
Atif Yilmaz,2,35473,8.55
C. Prem Kumar,2,54490,8.45
Ron Fricke,2,80978,8.45
Zeki Ökten,2,36376,8.4
Dragan Bjelogrlic,2,26316,8.4
Moustapha Akkad,3,81883,8.4
Lee Unkrich,2,1572720,8.35
Jeethu Joseph,2,88453,8.35
Mari Selvaraj,2,43785,8.35


##🔎 O que observei
A tabela acima mostra as melhores avaliações entre os diretores com o menor número de filmes produzidos.

Fiz um filtro para considerar apenas diretores com ao menos 2 filmes produzidos, e no máximo 5. Também tinham que ter o mínimo de 10 mil avaliações recebidas.

Essa análise pode complementar também o estudo das 3 primeiras perguntas, já que mostra um circuito "off" de cinema, de diretores que estão fora do mainstream, mas produzindo filmes aclamados.

- Obs: esses diretores não necessariamente trabalharam apenas nesses filmes, mas é o que consta na base de dados, nos filtros praticados.

### 🧠 Conclusão
São diretores Sérvios, Turcos e Indianos que não vão estar em qualquer cinema, mas que produziram filmes bem avaliados, e avaliados por muita gente. 

##9 - Gêneros de Cinema ao Longo das Décadas
Quais gêneros tiveram o maior número de títulos produzidos por década

In [0]:
%sql
WITH generos_por_decada AS (
  SELECT
    FLOOR(startYear / 10) * 10 AS decada,
    TRIM(genero) AS genero,
    COUNT(*) AS total_titulos
  FROM gold_title_basics
  LATERAL VIEW explode(split(genres, ',')) AS genero
  WHERE startYear >= 1910
    AND startYear <= 2020
    AND genres IS NOT NULL
    AND genres != '\\N'
    AND titleType = 'movie'
  GROUP BY FLOOR(startYear / 10) * 10, TRIM(genero)
),
ranking_generos AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY decada ORDER BY total_titulos DESC) AS posicao
  FROM generos_por_decada
)
SELECT decada, genero
FROM ranking_generos
WHERE posicao = 1
ORDER BY decada;


decada,genero
1910,Drama
1920,Drama
1930,Drama
1940,Drama
1950,Drama
1960,Drama
1970,Drama
1980,Drama
1990,Drama
2000,Drama


## 🎭 Gêneros Cinematográficos Predominantes por Década

## 🔎 Observações:

**Drama** reina absoluto por mais de um século como o gênero mais produzido, o que demonstra sua versatilidade e profundidade para abordar temas humanos, sociais e existenciais.

🎬 A primeira mudança só ocorre em **2020**, com o **Documentário** assumindo o posto. Isso pode estar relacionado ao crescimento de produções independentes, plataformas de streaming e um maior interesse por conteúdos informativos e reais, especialmente em um contexto pós-pandemia.

- obs: A permanência do drama como gênero dominante também pode indicar uma possível limitação na categorização dos dados, já que muitos títulos são classificados como "Drama" junto com outros gêneros.

##10 - 📺 Produção de Séries de TV nos Últimos 15 anos

In [0]:
%sql
SELECT
  startYear AS ano,
  COUNT(*) AS total_titulos
FROM gold_title_basics
WHERE titleType = 'tvSeries'
  AND startYear >= 2010
  AND startYear <= 2024
  AND startYear IS NOT NULL
GROUP BY startYear
ORDER BY startYear;


ano,total_titulos
2010,7086
2011,8558
2012,9351
2013,9750
2014,10192
2015,11167
2016,11393
2017,11611
2018,11641
2019,11484



![Gráfico da Pergunta 10](https://github.com/plemos66/projeto_mvp_eng_de_dados/raw/main/img/10.png)


## 📺 10. Evolução da Produção de Séries de TV (2010–2024)

---


- Entre **2010 e 2020**, o número de produções de séries cresceu de forma consistente, passando de cerca de **7 mil** para mais de **12 mil títulos por ano** — um crescimento de quase 80%.

- O pico ocorreu em **2020**, com **12.617** novas séries — possivelmente impulsionado pelo aumento de demanda por conteúdo digital durante a pandemia!

- A partir de **2021**, observa-se uma **queda progressiva**, chegando a **7.738 séries em 2024**, número comparável ao de 2010.

- Essa desaceleração pode estar relacionada à **saturação de mercado**, **contenção orçamentária de grandes estúdios**, ou mudanças no comportamento do consumidor com a diversificação de formatos (minisséries, podcasts, canais curtos como TikTok etc.).

